# Imports

In [9]:
import pandas as pd
import numpy as np

import world_trade_data as wits

from tqdm import tqdm

In [2]:
# df_import_share = wits.get_indicator('MPRT-PRTNR-SHR', reporter=reporter, year=year, name_or_id='name', partner='all').reset_index().sort_values(by='Value', ascending=False)
# df_export_share = wits.get_indicator('XPRT-PRTNR-SHR', reporter=reporter, year=year, name_or_id='name', partner='all').reset_index().sort_values(by='Value', ascending=False)

In [3]:
# df_import_share = df_import_share.query("Partner not in @regions")[['Partner', 'Value']]
# df_export_share = df_export_share.query("Partner not in @regions")[['Partner', 'Value']]

# Initialise Variables and Parameters

In [11]:
countries = wits.get_countries()

regions = list(countries[countries.grouptype == 'Region']['name'].values)
regions.extend([region.replace('&', 'and') for region in regions if '&' in region])

countries = countries[countries.grouptype != 'Region']

year = '2019'
error_countries = []

# Collect Trade Data (and Write to Disk)

In [12]:
for reporter in tqdm(countries.index.values):
    try:
                
        df_import = wits.get_indicator('MPRT-TRD-VL', reporter=reporter, year=year, name_or_id='name', partner='all').reset_index()
        df_export = wits.get_indicator('XPRT-TRD-VL', reporter=reporter, year=year, name_or_id='name', partner='all').reset_index()

        df_import = df_import.groupby('Partner').sum().sort_values(by = 'Value', ascending=False).reset_index()
        df_export = df_export.groupby('Partner').sum().sort_values(by = 'Value', ascending=False).reset_index()

        df_import = df_import.query("Partner not in @regions")
        df_export = df_export.query("Partner not in @regions")

        df = pd.concat([df_import, df_export]).groupby('Partner')['Value'].sum().reset_index().set_index('Partner')

        df.to_csv(f'Country Trade Data/{reporter}.csv')
    except:
        error_countries.append(reporter)
    

100%|██████████| 258/258 [14:40<00:00,  3.41s/it]


# Write Errors to Disk

In [18]:
with open('error_countries.npy', 'wb') as f:
    np.save(f, np.array(error_countries))

# Collate Data

In [45]:
import os
final_df = pd.DataFrame()

In [47]:
for subdir, dirs, files in os.walk('Country Trade Data'):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith(".csv"):
            df = pd.read_csv(filepath)
            df = df.rename(columns={"Partner":"Target", "Value":"Weight"})
            df['Source'] = countries.loc[file.split('.')[0]]['name']
            df = df[['Source','Target', 'Weight']]
            final_df = pd.concat([final_df, df], ignore_index=True)
            # print (filepath)
final_df

,Source,Target,Weight
0,Aruba,Brazil,9.922368e+04
1,Aruba,China,1.807202e+05
2,Aruba,Colombia,1.517809e+05
3,Aruba,Curaçao,1.266897e+05
4,Aruba,"Korea, Rep.",6.276485e+04
...,...,...,...
28786,Zimbabwe,United States Minor Outlying I,8.586520e+01
28787,Zimbabwe,Unspecified,3.083542e+06
28788,Zimbabwe,Uruguay,1.914964e+03
28789,Zimbabwe,Vietnam,1.305147e+03


In [50]:
final_df.to_csv('edges.csv', index=False)